In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 1.8MB 15.2MB/s 
     |████████████████████████████████| 133kB 52.2MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 174kB 54.2MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=e63331f5a5fe4d285d5f9244e962d562796a91be4750173367c2e0b3c286f2a3
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=023105442e1738da57b821f4da29a572508c65677b2595b0c743ad3e235b6a87
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [5]:
from model.CvT import CvT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [6]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [8]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 64,
    "patch_size" : 16,
    "num_classes" : 100,
    "dim" : 128,
    "depth" : 12,
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.1,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar100_ViT", entity='NoguNogu',name='CvT')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms


train_transforms = transforms.Compose([transforms.Resize((config['image_size'], config['image_size'])),
                                       transforms.RandomResizedCrop(config['image_size']),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()
                                       ])

val_transforms = transforms.Compose([
                                     transforms.Resize((config['image_size'], config['image_size'])),
                                     transforms.RandomResizedCrop(config['image_size']),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor()
                                     ])


trainset = datasets.CIFAR100(
    root='./data/Cifar100',
    train=True,
    download=False,
    transform=train_transforms)
    
testset = datasets.CIFAR100(
    root='./data/Cifar100',
    train=False,
    download=False,
    transform=val_transforms)

trainloader = DataLoader(trainset,
                         batch_size=config['batch_size'],
                         shuffle=True,
                         num_workers=2)

testloader = DataLoader(testset,
                         batch_size=config['batch_size'],
                         shuffle=False,
                         num_workers=2)

In [10]:
model = CvT(
    num_classes= config['num_classes']
    ).to(device)

In [11]:
from torchsummary import summary
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,472
         LayerNorm-2           [-1, 16, 16, 64]             128
            Conv2d-3           [-1, 64, 16, 16]             576
       BatchNorm2d-4           [-1, 64, 16, 16]             128
            Conv2d-5           [-1, 64, 16, 16]           4,096
   DepthWiseConv2d-6           [-1, 64, 16, 16]               0
            Conv2d-7             [-1, 64, 8, 8]             576
       BatchNorm2d-8             [-1, 64, 8, 8]             128
            Conv2d-9            [-1, 128, 8, 8]           8,192
  DepthWiseConv2d-10            [-1, 128, 8, 8]               0
          Softmax-11              [-1, 256, 64]               0
           Conv2d-12           [-1, 64, 16, 16]           4,160
          Dropout-13           [-1, 64, 16, 16]               0
        Attention-14           [-1, 64,

In [12]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [13]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 3.9667 - acc: 0.0923 - val_loss : 3.7854 - val_acc: 0.1173




Epoch : 2 - loss : 3.6526 - acc: 0.1425 - val_loss : 3.5842 - val_acc: 0.1555




Epoch : 3 - loss : 3.4951 - acc: 0.1669 - val_loss : 3.4564 - val_acc: 0.1791




Epoch : 4 - loss : 3.3712 - acc: 0.1907 - val_loss : 3.3658 - val_acc: 0.1930




Epoch : 5 - loss : 3.2769 - acc: 0.2079 - val_loss : 3.3040 - val_acc: 0.2093



...


Epoch : 16 - loss : 2.7640 - acc: 0.3053 - val_loss : 2.9170 - val_acc: 0.2848




Epoch : 17 - loss : 2.7247 - acc: 0.3128 - val_loss : 2.8982 - val_acc: 0.2823




Epoch : 18 - loss : 2.6969 - acc: 0.3171 - val_loss : 2.8929 - val_acc: 0.2933




Epoch : 19 - loss : 2.6722 - acc: 0.3234 - val_loss : 2.8626 - val_acc: 0.2922




Epoch : 20 - loss : 2.6329 - acc: 0.3338 - val_loss : 2.8388 - val_acc: 0.3029



In [14]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun May 16 07:14:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    51W / 300W |   2837MiB / 16160MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------